In [2]:
import matplotlib.pyplot as plt
import mpmath
import numpy as np
%matplotlib notebook
%load_ext autoreload
%autoreload 2
from qiskit.ignis.verification import state_tomography_circuits, StateTomographyFitter
from qiskit.tools import job_monitor
from qiskit.providers.aer.noise import NoiseModel
from hdf5_reader import Hdf5Reader
from qiskit.quantum_info import Statevector
from Tomography import tomography
import Tomography.tomography
import qiskit.quantum_info as qi

from basic_decomposer import BasicDecomposer
from qiskit import *
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
#IBMQ.save_account(
#    '280beccbee94456a161a6cbc217e1366bc278bf60e22bd30281fa0ca5bec6e50897278ef818f3c53f6700e04b9ed32ea364195044413b7e02836a79d886b03d9',
#    overwrite=True)
#IBMQ.load_account()
#provider = IBMQ.get_provider(hub='ibm-q-research')


In [3]:
import pickle

with open('simulator_job_6q_real.pkl', 'rb') as handle:
    job_simulator = pickle.load(handle)
print(job_simulator)

[{'000000': 10.0, '000001': 9.0, '000010': 5.0, '000011': 16.0, '000100': 6.0, '000101': 9.0, '000110': 12.0, '000111': 16.0, '001000': 12.0, '001001': 6.0, '001010': 21.0, '001011': 29.0, '001100': 9.0, '001101': 24.0, '001110': 16.0, '001111': 43.0, '010000': 10.0, '010001': 10.0, '010010': 9.0, '010011': 15.0, '010100': 8.0, '010101': 12.0, '010110': 23.0, '010111': 18.0, '011000': 2.0, '011001': 3.0, '011010': 2.0, '011011': 6.0, '011100': 2.0, '011101': 4.0, '011110': 7.0, '011111': 6.0, '100000': 1.0, '100001': 4.0, '100010': 4.0, '100100': 3.0, '100101': 1.0, '100110': 4.0, '100111': 3.0, '101000': 3.0, '101010': 6.0, '101011': 7.0, '101100': 3.0, '101101': 11.0, '101110': 6.0, '101111': 5.0, '110000': 1.0, '110001': 9.0, '110010': 9.0, '110011': 9.0, '110100': 9.0, '110101': 15.0, '110110': 24.0, '110111': 15.0, '111000': 8.0, '111001': 8.0, '111010': 9.0, '111011': 4.0, '111100': 10.0, '111101': 2.0, '111110': 16.0, '111111': 11.0}, {'000000': 10.0, '000001': 8.0, '000010': 4.

In [4]:
reader1 = Hdf5Reader("new_2layers_6qubits.h5").get_adapter()
decomposer = BasicDecomposer(reader1)
circuit1 = decomposer.decompose(barrier=True)
reader2 = Hdf5Reader("new_2layers_6qubits_good.h5").get_adapter()
circuit2 = BasicDecomposer(reader2).decompose(barrier=True)

## $$\huge \text{Shot Reduction}$$

In [16]:
import copy
import random

def reduce_shots(results, reduce):
    randomlist = random.sample(range(1, 590), reduce)
    new_res = copy.deepcopy(results)

    for result in new_res:
        balls = [a for a in result.values()]
        bounds = [b for b in result.keys()]
        takeout = [0 for b in bounds]
        for cell in randomlist:
            n = 0
            index = 0
            while n + balls[index] <= cell :
                n += balls[index]
                index += 1
            takeout[index] += 1
        for i in range(len(bounds)-1):
            result[bounds[i]] -= takeout[i]
    return new_res


In [5]:
nums = range(0,720,20)
#x = [2**a for a in nums]
fidelities = []
errors = []
for i in nums:
    fid = []
    for i in range(10):
        red_res = reduce_shots(job_simulator, i)
        matrix_sim_red = Tomography.tomography.get_density_matrix(red_res,0);
        fid.append(tomography.calc_fidelity(matrix_sim_red,Statevector(reader1.get_psi())))
    fidelities.append(sum(fid)/len(fid))
    errors.append((np.std(fid)))


0
0
0
0
0
0
0
0
0
0
0


KeyboardInterrupt: 

In [17]:
nums = range(0,720,20)
#x = [2**a for a in nums]
errors = []
fid = []
for i in range(10):
    red_res = reduce_shots(job_simulator, 400)
    matrix_sim_red = Tomography.tomography.get_density_matrix(red_res,0);
    a = tomography.calc_fidelity(matrix_sim_red,Statevector(reader1.get_psi()))
    fid.append(a)
    print(a)
fidelitiy = sum(fid)/len(fid)
errors.append((np.std(fid)))


0
(0.1617095373962862+1.734723475976807e-18j)
0
(0.15765104898532722+1.734723475976807e-18j)
0
(0.161062671957642-5.204170427930421e-18j)
0
(0.15312324476028424-1.734723475976807e-18j)
0
(0.15782372763523927-1.734723475976807e-18j)
0
(0.15516229401744153+1.734723475976807e-18j)
0
(0.16601450339225243+0j)
0
(0.15892703932704574+8.673617379884035e-19j)
0
(0.16050944847470033-1.734723475976807e-18j)
0
(0.15147720562919187+0j)


In [12]:
print(fidelitiy)
print(errors)

(0.16650382091139157+2.168404344971009e-19j)
[0.003706255420429927]


In [131]:
%matplotlib notebook
from matplotlib import pyplot as plt
plt.errorbar(nums,fidelities,errors)
plt.legend()
plt.grid()
plt.show()

<IPython.core.display.Javascript object>

No handles with labels found to put in legend.


In [6]:
nums = range(0,721,20)

#x = [2**a for a in nums]
fidelities_raw = []
fidelities_raw_error = []
fidelities = []
fidelities_error = []
fidelities_fixed = []
fidelities_fixed_error = []

for i in nums:
    f = []
    raw = []
    fixed = []

    for j in range(20):
    #red_res = reduce_shots(job_simulator, i)
        matrix_sim_raw = Tomography.tomography.get_density_matrix(job_simulator,i)
        matrix_sim_fixed = Tomography.tomography.get_density_matrix(job_simulator,i) * (729/(729 - i))

        raw.append(tomography.calc_fidelity(matrix_sim_raw,Statevector(reader1.get_psi())))
        fixed.append(tomography.calc_fidelity(matrix_sim_fixed,Statevector(reader1.get_psi())) - i/(64*(729-i)))
        f.append(tomography.calc_fidelity(matrix_sim_fixed,Statevector(reader1.get_psi())))


    fidelities_raw.append(sum(raw)/len(raw))
    fidelities_raw_error.append((np.std(raw)))

    fidelities_fixed.append(sum(fixed)/len(fixed))
    fidelities_fixed_error.append((np.std(fixed)))

    fidelities.append(sum(f)/len(f))
    fidelities_error.append((np.std(f)))



0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
140
140
140
140
140
140
140
140
140
140
140
140
140
140
140
140
140
140
140
140
140
140
140
140
140
140
140
140
140
140


KeyboardInterrupt: 

In [ ]:
%matplotlib notebook

plt.errorbar(nums,fidelities,fidelities_error, label="With factor")
plt.errorbar(nums,fidelities_fixed,fidelities_fixed_error, label="With factor and fix Identity")
plt.errorbar(nums,fidelities_raw,fidelities_raw_error, label="Raw")
plt.legend()
plt.xlabel("Number of reduced circuits")
plt.ylabel("Fidelity")
plt.grid()
plt.show()

In [114]:
matrix_sim_raw = Tomography.tomography.get_density_matrix(job_simulator,0);
fid = tomography.calc_fidelity(matrix_sim_raw,Statevector(reader1.get_psi()))
print(fid)
print(job_simulator)

0


KeyboardInterrupt: 

In [115]:
print(job_simulator)



[{'000000': 10.0, '000001': 9.0, '000010': 5.0, '000011': 16.0, '000100': 6.0, '000101': 9.0, '000110': 12.0, '000111': 16.0, '001000': 12.0, '001001': 6.0, '001010': 21.0, '001011': 29.0, '001100': 9.0, '001101': 24.0, '001110': 16.0, '001111': 43.0, '010000': 10.0, '010001': 10.0, '010010': 9.0, '010011': 15.0, '010100': 8.0, '010101': 12.0, '010110': 23.0, '010111': 18.0, '011000': 2.0, '011001': 3.0, '011010': 2.0, '011011': 6.0, '011100': 2.0, '011101': 4.0, '011110': 7.0, '011111': 6.0, '100000': 1.0, '100001': 4.0, '100010': 4.0, '100100': 3.0, '100101': 1.0, '100110': 4.0, '100111': 3.0, '101000': 3.0, '101010': 6.0, '101011': 7.0, '101100': 3.0, '101101': 11.0, '101110': 6.0, '101111': 5.0, '110000': 1.0, '110001': 9.0, '110010': 9.0, '110011': 9.0, '110100': 9.0, '110101': 15.0, '110110': 24.0, '110111': 15.0, '111000': 8.0, '111001': 8.0, '111010': 9.0, '111011': 4.0, '111100': 10.0, '111101': 2.0, '111110': 16.0, '111111': 11.0}, {'000000': 0, '000001': 0, '000010': 0, '000